In [1]:
import pandas as pd
import geopandas as gpd
import geoparquet as gpq

import itertools
import numpy as np
from shapely import wkt

In [2]:
def convert_geopackage_to_geoparquet(geo_package, key = 'ID', output_path = None):
    geo_package.to_crs("EPSG:4326", inplace=True)
    geo_package['wkt'] = geo_package['geometry'].to_wkt()
    
    geo_dataframe = gpd.GeoDataFrame()
    geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])    
    geo_dataframe = gpd.GeoDataFrame(geo_dataframe, geometry="geometry")
    geo_dataframe[key] = geo_package[key]
    
    return geo_dataframe.merge(geo_package[[i for i in geo_package.columns if i not in ["geometry", "wkt"]]],
                              left_on=key, right_on = key)


In [15]:
vol_gdf = gpd.read_file("../inputs/msm_zones/Sectors.gpkg", engine='pyogrio')
vol_gdf = vol_gdf[['Sector_3', 'LBA_Name', 'geometry']].rename(columns={'Sector_3': 'Sector'})
vol_parquet = convert_geopackage_to_geoparquet(vol_gdf, key='Sector')
vol_parquet.fillna(0, inplace=True)
vol_parquet.to_parquet('../inputs/msm_zones/sectors.parquet', compression = 'zstd')

/var/folders/4x/lf34fbnd6sjcsfz9532hs7yrlpdrcj/T/ipykernel_33261/232571482.py:6: FutureWarning: You are adding a column named 'geometry' to a GeoDataFrame constructed without an active geometry column. Currently, this automatically sets the active geometry column to 'geometry' but in the future that will no longer happen. Instead, either provide geometry to the GeoDataFrame constructor (GeoDataFrame(... geometry=GeoSeries()) or use `set_geometry('geometry')` to explicitly set the active geometry column.
  geo_dataframe["geometry"] = gpd.GeoSeries.from_wkt(geo_package["wkt"])


In [5]:
pd.read_parquet('data.parquet')

,ID,@vcv_26116,@vcv_26150,geometry
0,1000-5000,0.785221,0.770799,"b'\x01\x02\x00\x00\x00\x02\x00\x00\x00]\xf9,\x..."
1,1001-1002,0.174206,0.180266,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xe5\x9a...
2,1002-1001,0.161362,0.161761,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00#\xbc=\x...
3,1002-1013,0.191935,0.196301,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00#\xbc=\x...
4,1002-1863,0.108816,0.109032,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00#\xbc=\x...
...,...,...,...,...
15354,14002-1132,0.000000,0.000000,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xcc\xee...
15355,14002-14001,0.000000,0.000000,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xcc\xee...
15356,5242-7334,0.000000,0.000000,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00_)\xcb\x...
15357,5340-7334,0.000000,0.000000,b'\x01\x02\x00\x00\x00\x02\x00\x00\x00\xeb\x8e...
